In [1]:
%load_ext autoreload

In [2]:
from certified_iris_generator import CertifiedIrisRegionGenerator
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from sandbox import rrtiris

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RigidTransform, RollPitchYaw
import meshcat
from pydrake.all import GenerateSeedingPolytope
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.multibody.rational_forward_kinematics import FindEpsilonLower, FindEpsilonLowerVector, FindEpsilonUpperVector
from pydrake.solvers import mathematicalprogram as mp

# Build Plant


In [4]:
q0 = [0.0, 0.0, 0.0]
q_low  = [-1.7, -2., -1.7]
q_high = [ 1.7,  2.,  1.7]


In [5]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/oneDOF_iiwa7_with_box_collision.sdf")
twoDOF_iiwa_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/twoDOF_iiwa7_with_box_collision.sdf")

In [6]:
box_asset = FindResourceOrThrow("drake/sos_iris_certifier/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(twoDOF_iiwa_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset))



locs = [[0.,0.,0.],[0.,.5,0.],[0.,-.5,0.]]
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", models[0]), RigidTransform(locs[0]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_twoDOF_link_0", models[1]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[1]))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), RigidTransform(RollPitchYaw([0,0, -np.pi/2]).ToRotationMatrix(), locs[2]))


plant.Finalize()

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
            
#q_low = plant.GetPositionLowerLimits().tolist()
#q_high = plant.GetPositionUpperLimits().tolist()

Ratfk = RationalForwardKinematics(plant)


In [7]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram
visualizer.visualize_collision_constraint(N = 30)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [8]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [9]:
#filter fused joints self collisions 
digaram = visualizer.diagram
context = visualizer.diagram_context
plant_context = visualizer.plant_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
iiwa_oneDOF_gids = [gid for gid in gids if "iiwa7_oneDOF::" in get_name_of_gid(gid)]
iiwa_twoDOF_gids = [gid for gid in gids if "iiwa7_twoDOF::" in get_name_of_gid(gid)]

oneDOF_fused_col_geom = iiwa_oneDOF_gids[2:]
iiwa_oneDOF_fused_set = GeometrySet(oneDOF_fused_col_geom)
twoDOF_fused_col_geom = iiwa_twoDOF_gids[4:]
iiwa_twoDOF_fused_set = GeometrySet(twoDOF_fused_col_geom)
# print([get_name_of_gid(gid) for gid in oneDOF_fused_col_geom])
# print([get_name_of_gid(gid) for gid in twoDOF_fused_col_geom])
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_oneDOF_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(iiwa_twoDOF_fused_set))
pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(len(inspector.GetCollisionCandidates()), "->", len(pairs))

150 -> 150
137 -> 150


In [10]:
print(iiwa_oneDOF_gids[-1])
print(iiwa_twoDOF_gids[-1])

<GeometryId value=140>
<GeometryId value=92>


# Setup IRIS Options and Generate Regions

In [11]:
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.0, -1.3, -1.3],  # start
#                         [0.9, -1.5, -0.8]     # goal
#                          ])
# # More interesting seed points
seed_points_q = np.array([[0.0, 0, 0], # zero config
                        [ 1.3,  2.0, -1.7],  # start: right over left
                        [ 0.9, -1.3, -1.0],  # goal: left over right
                        [-0.6, -0.9, -1.3],
                        [-0.3, -0.5, -1.7]   # passing
                         ])
# seed_points_q = np.array([[0.0, 0, 0], # zero config
#                         [0.8, 1.3, -0.8],  # START: blue low green up
#                         [0.1, 0.9, -1.2],     # GOAL: green low other up
#                         [0.2, 1.6, -0.6],
#                         [0.5, -1.9, -0.9]
#                          ])[:,(0,1,2)]   # passing


seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((3,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)

start = seed_points[1,:]
goal = seed_points[2,:]

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), np.zeros((3,)) ))
    
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])
context = diagram.CreateDefaultContext()
q_star = np.zeros(3)

In [12]:
# regions = []

# iris_options = IrisOptionsRationalSpace()
# iris_options.require_sample_point_is_contained = True
# iris_options.iteration_limit = 20
# iris_options.configuration_space_margin = 1e-5
# iris_options.max_faces_per_collision_pair = 60
# iris_options.termination_threshold = -1
# iris_options.q_star = np.zeros(3)
# iris_options.relative_termination_threshold = 0.05
# iris_options.enable_ibex = False
# #deprecated
# iris_options.certify_region_with_sos_during_generation = False
# iris_options.certify_region_with_sos_after_generation = False

# for i, s in enumerate(seed_points):
#     plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
#     if False:
#         #starting_hpolyhedron = regions[i-1]
#         r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
#                                               iris_options, starting_hpolyhedron)
#     else:
#         r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
#     regions.append(r)
#     print(f'Completed region: {i+1}/{len(seed_points)}')
#     print(f"Sample point contained = {np.all(r.A()@s <= r.b())}")
#     print(f"Sample point contained componentwise = {r.A()@s <= r.b()}")
#     print()

In [13]:
# if do_viz:
#     visualizer.plot_regions(regions, ellipses=None, region_suffix='original')

In [14]:
# regions_default = []
# max_vects = []
# min_vects = []
# num_perm_dim = 2
# num_rot = 2

# for i, s in enumerate(seed_points):
#     regions_default.append(GenerateSeedingPolytope(s, num_perm_dim, num_rot))
# #     eps_min_vect = FindEpsilonLowerVector(regions_default[i].A(), 
# #                                       regions_default[i].b(), 
# #                                       limits_t[0], limits_t[1], s)
# #     min_vects.append(eps_min_vect)
# #     eps_max_vect = FindEpsilonUpperVector(regions_default[i].A(), 
# #                                       regions_default[i].b(), 
# #                                       limits_t[0], limits_t[1])
# #     max_vects.append(eps_max_vect)
    
    

In [15]:
# regions_default_small = []
# regions_default_big = []
# # for i, r in enumerate(regions_default):
# #     regions_default_small.append(HPolyhedron(regions_default[i].A(), regions_default[i].b() + 0.75*min_vects[i]))
# #     regions_default_big.append(HPolyhedron(regions_default[i].A(), regions_default[i].b() + max_vects[i]))
    
# if do_viz:
#     visualizer.plot_regions(regions_default, ellipses=None, region_suffix='_default')
# #     visualizer.plot_regions(regions_default_small, ellipses=None, region_suffix='_small')
# #     visualizer.plot_regions(regions_default_big, ellipses=None, region_suffix='_big')

In [16]:
# cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
#                                    rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
#                                    rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
# filtered_collision_pairs = set()
# solver_options = mp.SolverOptions()
# solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

In [17]:
# vector_bisection_search_options_list = []
# scalar_bisection_search_options_list = []
# bilinear_alternation_options_list = []
# integrated_search_options_list = []


# search_tuples = []

# for i, r in enumerate(regions_default):
#     vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
#     vector_bisection_search_options.max_iters = 20
#     vector_bisection_search_options.max_feasible_iters = 5
#     vector_bisection_search_options.search_d = True
#     vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r.A(), r.b(),
#                                                              limits_t[0], limits_t[1], seed_points[i,:])
#     vector_bisection_search_options.epsilon_max = 0.25*FindEpsilonUpperVector(r.A(), 
#                                       r.b(), 
#                                       limits_t[0], limits_t[1])
#     vector_bisection_search_options_list.append(vector_bisection_search_options)
    
#     scalar_bisection_search_options = rational_forward_kinematics.BinarySearchOption()
#     scalar_bisection_search_options.epsilon_max = 1
#     scalar_bisection_search_options.max_iters = 15
#     scalar_bisection_search_options.search_d = True
#     scalar_bisection_search_options.epsilon_min = FindEpsilonLower(r.A(), r.b(),
#                                                          limits_t[0], limits_t[1],
#                                                          seed_points[i,:])
#     scalar_bisection_search_options_list.append(scalar_bisection_search_options)
    
    
#     bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()
#     bilinear_alternation_options.max_iters = 20
#     bilinear_alternation_options.lagrangian_backoff_scale = 1e-6
#     bilinear_alternation_options.polytope_backoff_scale = 1e-6
#     bilinear_alternation_options_list.append(bilinear_alternation_options)
    
#     integrated_search_options = rational_forward_kinematics.IntegratedRegionSearchOptions()
#     integrated_search_options.vector_bisection_search_options = vector_bisection_search_options
#     integrated_search_options.scalar_binary_search_options = scalar_bisection_search_options
#     integrated_search_options.bilinear_alternation_options = bilinear_alternation_options
#     integrated_search_options.max_method_switch = 2
#     integrated_search_options.use_vector_bisection_search = True
#     integrated_search_options_list.append(integrated_search_options)
    
#     search_tuples.append((r, integrated_search_options, seed_points[i,:]))

In [18]:
# def launch_integrated_search(region, integrated_search_options, seed_point):
#     return cspace_free_region.IntegratedCSpacePolytopeSearch(q_star,filtered_collision_pairs,
#                                                       region.A(), region.b(),
#                                                      integrated_search_options, 
#                                                       solver_options, seed_point
#                                                      )
# def launch_vector_bisection_search(region, integrated_search_options, seed_point):
#     return cspace_free_region.IntegratedCSpacePolytopeSearch(q_star,filtered_collision_pairs,
#                                                       region.A(), region.b(),
#                                                      integrated_search_options, 
#                                                       solver_options, seed_point
#                                                      )

In [19]:
# ret_vals = []
# for tup in search_tuples:
#     ret_vals.append(launch_integrated_search(*tup))

In [20]:
# i = 1
# r = regions[i]
# s = seed_points_q[i]
# vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
# vector_bisection_search_options.max_iters = 1
# vector_bisection_search_options.max_feasible_iters = 20
# vector_bisection_search_options.search_d = True
# vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r.A(), r.b(),
#                                                          limits_t[0], limits_t[1], seed_points[i,:])
# vector_bisection_search_options.epsilon_max = 0.98*vector_bisection_search_options.epsilon_min #np.zeros(r.b().shape[0])

In [21]:
# d_final = cspace_free_region.CspacePolytopeBisectionSearchVector(q_star,set(),
#                                                       r.A(), r.b(),
#                                                      vector_bisection_search_options, 
#                                                       solver_options, seed_points_q)

In [22]:
# r_new = HPolyhedron(r.A(), d_final)
# if do_viz:
#     visualizer.plot_regions([r_new], ellipses=None, region_suffix='_new')

In [23]:
# planes = cspace_free_region.separating_planes()

In [24]:
# planes[0].b

In [25]:
# #refine with CPP code:
# cpp_editted_scalar_regions = []
# for i in range(seed_points.shape[0]):
#     s = seed_points[i,:]
#     region_init = regions[i]
#     region_init, region = iris_scalar_handle(s, region_init)
#     cpp_editted_scalar_regions.append(region)
#     print(f"Completed {i+1}/{seed_points.shape[0]}")

In [26]:
# if do_viz:
#     visualizer.plot_regions(cpp_editted_regions, ellipses=None, region_suffix='cpp_editted_regions')

In [27]:
# #setup refine with python
# iris_kwargs = {
#     'iris_starting_ellipse_vol': 1e-5,
#     'iris_plane_pullback': 1e-4,
#     'iris_max_faces': -1
# }
# iris_generator = CertifiedIrisRegionGenerator(visualizer.diagram, plant, scene_graph, **iris_kwargs)
# regions, ellipses = iris_generator.iris_in_rational_space(seed_points)
# iris_generator.initalize_certifier(plane_order = 1, strict_pos_tol = 1e-4)

In [28]:
# if do_viz:
#     visualizer.plot_regions(iris_generator.regions, region_suffix = '_python_snopt')

In [29]:
# refine with python
#do_linesearch_cert = True
#if do_linesearch_cert:
#    iris_generator.certify_and_adjust_regions_by_linesearch(1e-5)

In [30]:
#if do_viz:
#    visualizer.plot_regions(iris_generator.linesearch_regions, region_suffix = '_new_by_linesearch')

In [31]:
from pydrake.all import Solve, eq, le, ge

#lam*(binner_new - pullback) - bouter<= -1e-10
def solve_pullback(b_outer, b_inner_new, lam, tol = 1e-9):
    prog = MathematicalProgram()
    pullback = prog.NewContinuousVariables(len(b_inner_new), 'pullback')
    prog.AddLinearConstraint(-lam, -np.inf*np.ones(len(b_outer)).reshape(-1,1) ,b_outer - tol - lam@b_inner_new, pullback)
    prog.AddBoundingBoxConstraint(0, np.inf, pullback)
    res = Solve(prog)
    pullback_sol = res.GetSolution(pullback)
    return res.is_success(), pullback_sol
    
def solve_hpoly_containment_problem(r_outer, r_inner):
    A_outer = r_outer.A()
    A_inner = r_inner.A()
    b_outer = r_outer.b()
    b_inner = r_inner.b()
    prog = MathematicalProgram()
    lam = prog.NewContinuousVariables(len(b_outer), len(b_inner), 'lam')
    prog.AddBoundingBoxConstraint(0.0, np.inf, lam)
    for col in range(A_outer.T.shape[1]):
        prog.AddLinearEqualityConstraint(A_inner.T, A_outer.T[:,col], lam.T[:,col])
        
    
    for r in range(lam.shape[0]):
        prog.AddLinearConstraint(b_inner[:,np.newaxis].T, np.array([-np.inf]), np.array([b_outer[r]- 1e-10]), lam[r, :])

    res =  Solve(prog)
    lam_sol = res.GetSolution(lam)
    return res.is_success(), lam_sol

def solve_volume_maximization_step(r_outer, r_inner, lam_sol, seed_point):
    A_inner = r_inner.A()
    b_inner = r_inner.b()
    b_outer = r_outer.b()
    dim = A_inner.shape[1]
    print('here')
    prog = MathematicalProgram()
    d = prog.NewContinuousVariables(dim, 'd')
    C = prog.NewSymmetricContinuousVariables(dim, 'C')
    eps = prog.NewContinuousVariables(A_inner.shape[0], 'eps')
    prog.AddMaximizeLogDeterminantCost(C)
    for idx in range(len(b_inner)):
        z = prog.NewContinuousVariables(dim + 1)
        z[0] = b_inner[idx] + eps[idx] - A_inner[idx, :]@d
        z[1:] = C @ A_inner[idx, :].T
        prog.AddLorentzConeConstraint(z)  
    prog.AddLinearConstraint(lam_sol, -np.inf*np.ones_like(b_outer), b_outer - lam_sol@b_inner, eps)
    prog.AddLinearConstraint(np.eye(len(eps)), A_inner@seed_point - b_inner, np.inf*np.ones_like(eps), eps)
    #prog.AddConstraint(le(lam_sol@(b_inner + eps) - b_outer, 0))
    #prog.AddConstraint(le(A_inner@seed_point , b_inner + eps))
    prog.AddBoundingBoxConstraint(0.0, 4.0, eps)
    res = Solve(prog)
    
    eps_sol = res.GetSolution(eps)
    print('here2')
    print(eps_sol)
    new_reg = HPolyhedron(A_inner, b_inner + eps_sol)
    return res.is_success(), eps_sol, res.get_optimal_cost()


def inner_approx_HPoly_bilinear_alternation(r_outer, r_inner_seed, seed_point, maxit = 10):
    #check if contained at start
    if r_inner_seed.ContainedInOtherHPolyhedron(r_outer):
        r_inner = r_inner_seed
        past_regions = []
        for it in range(maxit):
            print('step', it)
            #get certificate of containment
            success, lam_sol = solve_hpoly_containment_problem(r_outer, r_inner)
            print('containment step: ', success, 'lambda min', np.min(lam_sol))
            if not success:
                print('terminated early, containment lost')
                return past_regions[-2]
            #use certificate to push out faces of inner polytope
            success, eps, vol_surrogate = solve_volume_maximization_step(r_outer, r_inner_seed, lam_sol, seed_point)
            print('growth step: ', success, 'volume surrogate: ', vol_surrogate)
            print('eps', eps)
            #update polytope
            #success, pullback = solve_pullback(r_outer.b(), r_inner_seed.b() + eps, lam_sol)
            #|print('pullback: ', success, 'pullback ineq', np.max(lam_sol@(r_inner_seed.b() + eps - pullback)-r_outer.b()))
            
            r_inner = HPolyhedron(r_inner.A(), r_inner_seed.b() + eps - 1e-10) #-pullback
            print(f'max inscribed ellipse volume: {r_inner.MaximumVolumeInscribedEllipsoid().Volume()}')
            past_regions.append(r_inner)
        return r_inner
    else:
        raise ValueError("Seed HPolytope not contained in outer Hpolytope")

In [32]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-6
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(3)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
def iris_handle(seed, domain):
    seed_q = Ratfk.ComputeQValue(seed, np.zeros((3,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), seed_q)
    region = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context),
                                              iris_options, domain)
    return region
starting_poly = HPolyhedron.MakeBox(limits_t[0], limits_t[1])

In [33]:
compare_seed = seed_points[3]
iris_reg = iris_handle(compare_seed, starting_poly)
sym_poly = GenerateSeedingPolytope(compare_seed, 2, 2)
eps_min_vect = FindEpsilonLowerVector(sym_poly.A(),
                                      sym_poly.b(),
                                      limits_t[0], limits_t[1], compare_seed)

visualizer.plot_regions([iris_reg], region_suffix = 'bil_iris')
visualizer.plot_regions([sym_poly], region_suffix = 'bil_sym')

In [34]:
small_sym_poly = HPolyhedron(sym_poly.A(), sym_poly.b() + .9 * eps_min_vect)
visualizer.plot_regions([small_sym_poly], region_suffix = 'bil_small_sym')

In [35]:
print(eps_min_vect)
print(small_sym_poly.MaximumVolumeInscribedEllipsoid().Volume())
print(small_sym_poly.PointInSet(compare_seed))
print(small_sym_poly.ContainedInOtherHPolyhedron(iris_reg))
print(iris_reg.PointInSet(compare_seed))

[-0.09999999 -0.09999999 -0.09999999 -0.10000001 -0.10000001 -0.10000001
 -0.09999999 -0.1        -0.09999999 -0.10000001 -0.1        -0.10000001
 -0.09999999 -0.09999999 -0.1        -0.10000001 -0.10000001 -0.1
 -0.09999999 -0.10000001 -0.09999999 -0.10000001 -0.09999999 -0.10000001
 -0.09999999 -0.09999999 -0.10000001 -0.10000001 -0.10000001 -0.09999999]
4.188788206486698e-06
True
True
True


In [36]:
r_new = inner_approx_HPoly_bilinear_alternation(iris_reg, small_sym_poly, compare_seed, maxit = 40)
visualizer.plot_regions([r_new], region_suffix = 'bil_new')
print(r_new.MaximumVolumeInscribedEllipsoid().Volume())
print(r_new.PointInSet(compare_seed))
print(r_new.ContainedInOtherHPolyhedron(iris_reg))

step 0
containment step:  True lambda min -2.3638250135071814e-08
here
here2
[ 2.05424939e+00  2.03562160e+00  2.06035640e+00  1.99135493e+00
  1.69468570e-01  1.97048143e+00  2.04994209e-01  1.87476452e-02
  2.06035640e+00  1.46428120e-01  7.97865936e-01 -1.03388775e-09
  2.05424939e+00  2.75665471e-01  1.85194410e-01  1.99135493e+00
 -5.39915857e-10  3.12564241e-01  2.04325360e+00  1.99535008e+00
  2.06035640e+00  2.00600068e+00  1.67857783e-01  1.97048143e+00
  2.05424939e+00  2.06425905e+00  2.02474876e+00  1.99135493e+00
  1.97026687e+00  2.02192061e+00]
growth step:  True volume surrogate:  6.605465255397837
eps [ 2.05424939e+00  2.03562160e+00  2.06035640e+00  1.99135493e+00
  1.69468570e-01  1.97048143e+00  2.04994209e-01  1.87476452e-02
  2.06035640e+00  1.46428120e-01  7.97865936e-01 -1.03388775e-09
  2.05424939e+00  2.75665471e-01  1.85194410e-01  1.99135493e+00
 -5.39915857e-10  3.12564241e-01  2.04325360e+00  1.99535008e+00
  2.06035640e+00  2.00600068e+00  1.67857783e-01 

containment step:  True lambda min -2.8441451360253514e-07
here
here2
[1.00897612 1.06470625 1.22007024 0.97979663 1.03600947 0.91510929
 0.45273077 0.20451759 0.67128068 0.34046906 0.23203117 0.12473931
 1.00897612 1.17803503 0.57437079 0.97979663 0.1906723  0.23131447
 1.07730757 0.10480559 1.22007024 0.38833965 0.1829432  0.12470271
 1.00897612 0.66476775 0.44866661 0.97979663 0.12630685 0.36376068]
growth step:  True volume surrogate:  3.626434597182648
eps [1.00897612 1.06470625 1.22007024 0.97979663 1.03600947 0.91510929
 0.45273077 0.20451759 0.67128068 0.34046906 0.23203117 0.12473931
 1.00897612 1.17803503 0.57437079 0.97979663 0.1906723  0.23131447
 1.07730757 0.10480559 1.22007024 0.38833965 0.1829432  0.12470271
 1.00897612 0.66476775 0.44866661 0.97979663 0.12630685 0.36376068]
max inscribed ellipse volume: 0.11146741015753843
step 10
containment step:  True lambda min -2.7755575615628914e-17
here
here2
[1.0501162  1.10464651 0.90467101 1.00027901 1.08823906 0.94585348
 0.

containment step:  True lambda min -5.551115123125783e-17
here
here2
[1.01615706 1.05903559 0.75897776 0.9830967  1.07589681 0.91745408
 0.46313487 0.18752444 1.27371881 0.43830285 0.23818328 0.14603268
 1.01615706 1.00764543 0.67462209 0.9830967  0.22928432 0.22972362
 0.47375365 0.11895985 1.27371881 0.47341094 0.17765351 0.14465634
 1.01615706 0.7431921  0.53171333 0.9830967  0.15364924 0.3669685 ]
growth step:  True volume surrogate:  3.4377509909275394
eps [1.01615706 1.05903559 0.75897776 0.9830967  1.07589681 0.91745408
 0.46313487 0.18752444 1.27371881 0.43830285 0.23818328 0.14603268
 1.01615706 1.00764543 0.67462209 0.9830967  0.22928432 0.22972362
 0.47375365 0.11895985 1.27371881 0.47341094 0.17765351 0.14465634
 1.01615706 0.7431921  0.53171333 0.9830967  0.15364924 0.3669685 ]
max inscribed ellipse volume: 0.1346146316067478
step 20
containment step:  True lambda min -5.692771918058593e-07
here
here2
[1.03306758 1.08234932 0.75715328 1.01070994 1.09431092 0.93298246
 0.46

containment step:  True lambda min -1.3877787807814457e-17
here
here2
[1.04219747 1.09949584 0.84987936 1.01510709 1.13119226 0.93096394
 0.48698556 0.20373607 0.85121342 0.49063235 0.27239593 0.93096394
 1.04219747 0.69550702 0.79352928 1.01510709 0.23728358 0.26245107
 0.50895484 0.1051874  0.96723975 0.54691767 0.16255111 0.15033747
 1.04219747 0.81547352 0.63157172 1.01510709 0.1531013  0.39621351]
growth step:  True volume surrogate:  3.323189795928081
eps [1.04219747 1.09949584 0.84987936 1.01510709 1.13119226 0.93096394
 0.48698556 0.20373607 0.85121342 0.49063235 0.27239593 0.93096394
 1.04219747 0.69550702 0.79352928 1.01510709 0.23728358 0.26245107
 0.50895484 0.1051874  0.96723975 0.54691767 0.16255111 0.15033747
 1.04219747 0.81547352 0.63157172 1.01510709 0.1531013  0.39621351]
max inscribed ellipse volume: 0.15095434508445868
step 30
containment step:  True lambda min -2.7755575615628914e-17
here
here2
[1.03437122 1.09579915 0.84987992 1.01226014 1.12675945 0.92683696
 0.

0.15095389335997228
True
True


In [37]:
#certify found region
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

vector_bisection_search_options = rational_forward_kinematics.VectorBisectionSearchOption()
vector_bisection_search_options.max_iters = 3
vector_bisection_search_options.max_feasible_iters = 1
vector_bisection_search_options.search_d = True
filtered_collision_pairs = set()
vector_bisection_search_options.epsilon_min = FindEpsilonLowerVector(r_new.A(), r_new.b(),
                                                         limits_t[0], limits_t[1], compare_seed)

vector_bisection_search_options.epsilon_max = 0.25*FindEpsilonUpperVector(r_new.A(), 
                                      r_new.b(), 
                                      limits_t[0], limits_t[1],)
solver_options = mp.SolverOptions()
solver_options.SetOption(mp. CommonSolverOption.kPrintToConsole, 1)

b_feasible, separating_planes = cspace_free_region.CspacePolytopeBisectionSearchVector(
                 q_star, filtered_collision_pairs, r_new.A(), r_new.b(),
                 vector_bisection_search_options, solver_options)
r0_grown_vector = HPolyhedron(r_new.A(), b_feasible) 

[2022-02-06 22:09:42.073] [console] [info] Solver time 3.6581928730010986
[2022-02-06 22:09:48.056] [console] [info] Solver time 1.7042369842529297
[2022-02-06 22:09:55.349] [console] [info] Solver time 3.1300251483917236
[2022-02-06 22:09:55.401] [console] [info] total iter=1, feasible iter = 0
[2022-02-06 22:10:07.365] [console] [info] bilinear alt on d succeeded
[2022-02-06 22:10:07.390] [console] [info] Solver time 1.9592809677124023
[2022-02-06 22:10:07.451] [console] [info] total iter=2, feasible iter = 1
[2022-02-06 22:10:19.753] [console] [info] bilinear alt on d succeeded
[2022-02-06 22:10:19.779] [console] [info] Solver time 2.1730740070343018
[2022-02-06 22:10:19.843] [console] [info] total iter=3, feasible iter = 2


In [39]:
visualizer.plot_regions([r0_grown_vector], region_suffix = 'bil_cert')

In [43]:
import visualizations_utils
visualizer.region_to_separating_planes_dict = {r0_grown_vector: separating_planes}
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

plane_test = separating_planes[125]
print(plane_test.geometryA.get_value())
print(plane_test.geometryB.get_value())

def showres_verts(q):
    visualizer.showres(q)
   
    t = Ratfk.ComputeTValue(q, visualizer.q_star)
    vert_A = plane_test.positive_side_polytope.p_BV()[:, 0]
    vert_B = plane_test.negative_side_polytope.p_BV()[:, 0]
    geomA = plane_test.positive_side_polytope.get_id()
    geomB = plane_test.negative_side_polytope.get_id()
    #Ratfk.CalcLinkPoses(visualizer.q_star, plant.world_body().index())
    
    #print('A: ', plane_test.positive_side_polytope.body_index())
    #print('B: ', plane_test.negative_side_polytope.body_index())
    
    b = plane_test.b
    a = plane_test.a()
    b_eval_frame = b.Evaluate(dict(zip(b.GetVariables(), t)))
    a_eval_frame = np.array([a_idx.Evaluate(dict(zip(a_idx.GetVariables(), t))) for a_idx in a])
    
    X_EW = plant.GetBodyFromFrameId(plant.GetBodyFrameIdIfExists(plane_test.expressed_link))\
                .body_frame().CalcPoseInWorld(plant_context).inverse()
    X_WE = X_EW.inverse()
    
    R_EW = X_EW.rotation().matrix()
    V_EW = X_EW.translation()
    
    a_eval = a_eval_frame #a_eval_frame@R_EW
    b_eval = b_eval_frame #b_eval_frame - a_eval_frame@V_EW
    
    X_WA = plant.GetBodyFromFrameId(plant.GetBodyFrameIdIfExists(plane_test.positive_side_polytope.body_index()))\
                .body_frame().CalcPoseInWorld(plant_context)
    vert_A = X_WA@vert_A
    
    X_WB = plant.GetBodyFromFrameId(plant.GetBodyFrameIdIfExists(plane_test.negative_side_polytope.body_index()))\
                .body_frame().CalcPoseInWorld(plant_context)
    vert_B = X_WB@vert_B
    
    verts_tf_E = visualizer.transform(a_eval, b_eval, X_EW@vert_A, X_EW@vert_B, visualizer.plane_verts)
    verts_tf = (X_WE@verts_tf_E.T).T
    
    
    p_expr = X_EW.inverse()@np.zeros((3,))
    mat = meshcat.geometry.MeshLambertMaterial(color=visualizations_utils.rgb_to_hex((255,0,0)), wireframe=False)
    mat.opacity = 1.0
    
    
    #print(X_WA.translation())
    viz_utils.plot_point(loc=vert_A, radius=0.1, mat=mat, 
                         vis=visualizer.vis["bodyvert"][f"{geomA.get_value()}"],
                         marker_id='p1')
    
    viz_utils.plot_point(loc=vert_B, radius=0.1, mat=mat, 
                         vis=visualizer.vis["bodyvert"][f"{geomB.get_value()}"],
                         marker_id='p2')
    mat = meshcat.geometry.MeshLambertMaterial(color=visualizations_utils.rgb_to_hex((255,255,0)), wireframe=False)
    mat.opacity = 1.0
    viz_utils.plot_point(loc=p_expr, radius=0.2, mat=mat, 
                         vis=visualizer.vis["bodyvert"]["expressed"],
                         marker_id='expressed')
    if r0_grown_vector.PointInSet(t):
        mat = meshcat.geometry.MeshLambertMaterial(color=visualizations_utils.rgb_to_hex((0,255,0)), wireframe=False)
    else:
        mat = meshcat.geometry.MeshLambertMaterial(color=visualizations_utils.rgb_to_hex((255,0,0)), wireframe=False)
    mat.opacity = 0.5
    visualizer.vis['bodyvert']["plane"][f"{geomA.get_value()}, {geomB.get_value()}"].set_object(
            meshcat.geometry.TriangularMeshGeometry(verts_tf, visualizer.plane_triangles),
            mat)
    
q = q0.copy()

def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres_verts(q)
    
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

visualizer.jupyter_cell()

92
140


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [28]:
#for finding planes
geom_pairs_inspector = []
for idx, (geomA, geomB) in enumerate(inspector.GetCollisionCandidates()):
    geom_pairs_inspector.append((geomA, geomB))
    if geomA.get_value() == 92:
        print(idx, (geomA, geomB))

1 (<GeometryId value=92>, <GeometryId value=122>)
32 (<GeometryId value=92>, <GeometryId value=110>)
41 (<GeometryId value=92>, <GeometryId value=140>)
71 (<GeometryId value=92>, <GeometryId value=128>)
100 (<GeometryId value=92>, <GeometryId value=98>)
104 (<GeometryId value=92>, <GeometryId value=116>)
108 (<GeometryId value=92>, <GeometryId value=134>)
135 (<GeometryId value=92>, <GeometryId value=104>)
